<a href="https://colab.research.google.com/github/stefan-gietl/heat-exchangers/blob/main/1_Zielwert_kA_Bestimmung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zielwert kA
- 1 Imports
- 2 Vorgaben
- 3 Lösung
- 4 Ausgabe

## 1 Imports
Temperaturabhängige Stoffdaten werden von CoolProp (Teil der pyfluids-Bibliothek) bereitgestellt.

Die math-Bibliothek wird später für die Exponentialfunktion benötigt: math.exp(x) = $e^x$

In [ ]:
!pip install pyfluids
from CoolProp.CoolProp import PropsSI
import math

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.1/250.1 kB 5.1 MB/s eta 0:00:00


## 2 Vorgaben
Zunächst werden alle bekannten Vorgaben definiert.

In [ ]:
Tein_Heiss = 450  # °C
Tein_Kalt = 25  # °C
V_N_Heiss = 559  # Nm³/h
V_N_Kalt = 854  # Nm³/h
Taus_Heiss_Soll = 113  # °C
Taus_Kalt_Soll = 250  # °C

# Pressure Conditions
Druck_Heiss = 101575
Druck_Kalt = 101575

# Initial Values
Taus_Heiss = 200  # °C
Taus_Kalt = 200  # °C

# Initial k-Guess
k = 12  # W/m²K

A = 30  # m²

rho_norm = PropsSI("D", "T", 273.15, "P", 101325, "air")


## 3 Iteration

In [ ]:
error = 1
while abs(error) > 0.01:
    Tmittel_Heiss = (Tein_Heiss + Taus_Heiss) / 2
    Tmittel_Kalt = (Tein_Kalt + Taus_Kalt) / 2
    cp1 = PropsSI("C", "T", (Tmittel_Heiss + 273.15), "P", Druck_Heiss, "air")
    cp2 = PropsSI("C", "T", (Tmittel_Kalt + 273.15), "P", Druck_Kalt, "air")
    NTU1 = k * A / cp1 / (rho_norm * V_N_Heiss / 3600)
    NTU2 = k * A / cp2 / (rho_norm * V_N_Kalt / 3600)
    R1 = cp1 * V_N_Heiss / cp2 / V_N_Kalt
    R2 = 1 / R1
    eps1 = (1 - math.exp((R1 - 1) * NTU1)) / (1 - R1 * math.exp((R1 - 1) * NTU1))
    eps2 = (1 - math.exp((R2 - 1) * NTU2)) / (1 - R2 * math.exp((R2 - 1) * NTU2))

    # Temperatures
    Taus_Heiss = Tein_Heiss - eps1 * (Tein_Heiss - Tein_Kalt)
    Taus_Kalt = Tein_Kalt + eps2 * (Tein_Heiss - Tein_Kalt)
    Tmittel_Heiss = (Tein_Heiss + Taus_Heiss) / 2
    Tmittel_Kalt = (Tein_Kalt + Taus_Kalt) / 2

    # Correction Loop
    error = Taus_Heiss_Soll - Taus_Heiss
    k -= 0.01 * error

## 4 Outputs

In [ ]:
print(f"Zielwert gefunden k = {k:.4f}")
print(f"Taus_Heiss = {Taus_Heiss:.2f} °C")
print(f"Taus_Kalt = {Taus_Kalt:.2f} °C")

Leistung_R = cp1 * rho_norm * V_N_Heiss * (Tein_Heiss - Taus_Heiss) / 1000 / 3600  # kW
Leistung_L = cp2 * rho_norm * V_N_Kalt * (Taus_Kalt - Tein_Kalt) / 1000 / 3600  # kW

Zielwert gefunden k = 17.2655
Taus_Heiss = 113.01 °C
Taus_Kalt = 251.12 °C
